# Support for Workflow Automation

This notebook walks through existing functionality for (meta)data contributors to

1. *register workflows*, to

2. *install sensor logic* for automated workflow execution, to

3. *programmatically register workflow state transitions*, i.e. run events, and to

3. *programmatically register generated assets*, i.e. data and metadata outputs, with any workflow run event.

## Register workflows

In the `nmdc_runtime.api.boot.workflows` module, add an entry for your workflow to the `_raw` list. Examples:

```python
# nmdc_runtime/api/boot/workflows.py
    {
        "id": "test",
        "created_at": datetime(2021, 9, 9, tzinfo=timezone.utc),
        "name": "A test workflow",
        "description": "For use in unit and integration tests",
    },   
    {
        "id": "metadata-in-1.0.0",
        "created_at": datetime(2021, 10, 12, tzinfo=timezone.utc),
        "name": "general metadata ETL",
        "description": "Validate and ingest metadata from JSON files",
    },
    {
        "id": "apply-changesheet-1.0.0",
        "created_at": datetime(2021, 9, 30, tzinfo=timezone.utc),
        "name": "apply metadata changesheet",
        "description": "Validate and apply metadata changes from TSV/CSV files",
    },
    {
        "id": "export-study-biosamples-as-csv-1.0.0",
        "created_at": datetime(2022, 6, 8, tzinfo=timezone.utc),
        "name": "export study biosamples metadata as CSV",
        "description": "Export study biosamples metadata as CSV",
    },    
```

That's it. The `id` field is a primary key under administration by workflow authors. That is, it is up to those who register a workflow by `id` here to ensure that it corresponds to a semantically invariant (minor and patch updates may vary if no `-x.y.z` suffix is part of the registered `id`) version of an unambiguously known workflow. Concretely, there is no requirement for e.g. a commmit-hash-including GitHub link to the workflow's entrypoint.

## Install sensor logic

In the `nmdc_runtime.site.repository` module, you may add a function decorated with `dagster.sensor` (i.e. `@sensor` preceding the function's `def`), following the examples already installed.

Alternatively, if your workflow needs to run if and only if a new data object of a certain type is detected by the runtime, then you may declaratively hook into the existing generic `nmdc_runtime.site.repository.process_workflow_job_triggers` sensor by registering appropriate entries in the `_raw` lists of `nmdc_runtime.api.boot.triggers` and `nmdc_runtime.api.boot.object_types`. See the next subsection for details.

### Register object-type and trigger metadata

If your workflow needs to run if and only if a new data object of a certain type is detected by the runtime, you can add entries to two modules as per the following examples:

```python
# nmdc_runtime/api/boot/object_types.py
    {
        "id": "test",
        "created_at": datetime(2021, 9, 7, tzinfo=timezone.utc),
        "name": "A test object type",
        "description": "For use in unit and integration tests",
    },
    {
        "id": "metadata-in",
        "created_at": datetime(2021, 6, 1, tzinfo=timezone.utc),
        "name": "metadata submission",
        "description": "Input to the portal ETL process",
    },
    {
        "id": "metadata-changesheet",
        "created_at": datetime(2021, 9, 30, tzinfo=timezone.utc),
        "name": "metadata changesheet",
        "description": "Specification for changes to existing metadata",
    },    
```
```python
# nmdc_runtime/api/boot/triggers.py
    {
        "created_at": datetime(2021, 9, 9, tzinfo=timezone.utc),
        "object_type_id": "test",
        "workflow_id": "test",
    },
    {
        "created_at": datetime(2021, 6, 1, tzinfo=timezone.utc),
        "object_type_id": "metadata-in",
        "workflow_id": "metadata-in-1.0.0",
    },
    {
        "created_at": datetime(2021, 9, 30, tzinfo=timezone.utc),
        "object_type_id": "metadata-changesheet",
        "workflow_id": "apply-changesheet-1.0.0",
    },    
```

## Register workflow state transitions

## Register workflow-generated assets